###                                                Santader Product Recommendation System

#### Importing Necessary Libraries

In [1]:
import pickle
import gc
import xgboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# df = pd.read_csv('C:/Users/SAMRUDDHI/data/train_ver2.csv')

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Helper Functions 

In [20]:
months = ['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28', '2015-05-28',
         '2015-06-28', '2015-07-28', '2015-08-28', '2015-09-28', '2015-10-28',
         '2015-11-28', '2015-12-28', '2016-01-28', '2016-02-28', '2016-03-28',
         '2016-04-28', '2016-05-28']
       #[625457, 627394, 629209, 630367, 631957, 632110, 829817, 843201,
       #865440, 892251, 906109, 912021, 916269, 920904, 925076, 928274,
       #931453]
       
prods = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 
        'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 
        'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 
        'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
        'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
        'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
        'ind_nomina_ult1', 'ind_nom_pens_ult1',  'ind_recibo_ult1']
        
# 12 nonzero addedproducts (may16)
targetprods = ['ind_recibo_ult1', 'ind_cco_fin_ult1', 'ind_nom_pens_ult1',
    'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_ecue_fin_ult1',
    'ind_cno_fin_ult1', 'ind_ctma_fin_ult1', 'ind_reca_fin_ult1',
    'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_valo_fin_ult1']
    
#targetprods16 = ['ind_recibo_ult1', 'ind_cco_fin_ult1', 'ind_nom_pens_ult1',
#    'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_ecue_fin_ult1',
#    'ind_cno_fin_ult1', 'ind_ctma_fin_ult1', 'ind_reca_fin_ult1',
#    'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_valo_fin_ult1',
#    'ind_dela_fin_ult1', 'ind_deco_fin_ult1', 'ind_ctju_fin_ult1',
#    'ind_fond_fin_ult1']
#    
#targetprods14 = ['ind_recibo_ult1', 'ind_cco_fin_ult1', 'ind_nom_pens_ult1',
#    'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_ecue_fin_ult1',
#    'ind_cno_fin_ult1', 'ind_ctma_fin_ult1', 'ind_reca_fin_ult1',
#    'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_valo_fin_ult1',
#    'ind_dela_fin_ult1', 'ind_deco_fin_ult1']
      
# 22 nonzero added products (jun15)
targetprods22 = ['ind_cco_fin_ult1', 'ind_recibo_ult1', 'ind_nom_pens_ult1',
    'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_reca_fin_ult1', 
    'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_dela_fin_ult1',
    'ind_deco_fin_ult1', 'ind_ctma_fin_ult1', 'ind_fond_fin_ult1',
    'ind_ctop_fin_ult1', 'ind_valo_fin_ult1', 'ind_ctpp_fin_ult1',
    'ind_ctju_fin_ult1', 'ind_deme_fin_ult1', 'ind_plan_fin_ult1',
    'ind_cder_fin_ult1', 'ind_pres_fin_ult1', 'ind_hip_fin_ult1',
    'ind_viv_fin_ult1'] #, 'ind_aval_fin_ult1', 'ind_ahor_fin_ult1']
    
#targetprods12new = ['ind_cco_fin_ult1', 'ind_recibo_ult1', 'ind_nom_pens_ult1',
#    'ind_nomina_ult1', 'ind_tjcr_fin_ult1', 'ind_reca_fin_ult1', 
#    'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_dela_fin_ult1',
#    'ind_deco_fin_ult1', 'ind_ctma_fin_ult1', 'ind_fond_fin_ult1']
    
#targetprods = targetprods22
    
# Total added products (jun2015)
#ind_cco_fin_ult1     7565.0        ind_ctop_fin_ult1     182.0
#ind_recibo_ult1      7233.0        ind_valo_fin_ult1     125.0
#ind_nom_pens_ult1    6601.0        ind_ctpp_fin_ult1     117.0
#ind_nomina_ult1      4117.0        ind_ctju_fin_ult1      44.0
#ind_tjcr_fin_ult1    3807.0        ind_deme_fin_ult1      25.0
#ind_reca_fin_ult1    2325.0        ind_plan_fin_ult1      18.0
#ind_cno_fin_ult1     1573.0        ind_cder_fin_ult1       7.0
#ind_ecue_fin_ult1     962.0        ind_pres_fin_ult1       6.0
#ind_dela_fin_ult1     871.0        ind_hip_fin_ult1        4.0
#ind_deco_fin_ult1     413.0        ind_viv_fin_ult1        3.0
#ind_ctma_fin_ult1     278.0        ind_aval_fin_ult1       0.0
#ind_fond_fin_ult1     201.0        ind_ahor_fin_ult1       0.0

prodict = dict(zip(range(len(targetprods)),targetprods))
        
clientfeatures = ['fecha_dato', 'ncodpers', 'ind_empleado', 
       'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 
       'antiguedad', 'indrel', 'ult_fec_cli_1t', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
       'indfall', 'tipodom', 'cod_prov', 'nomprov',
       'ind_actividad_cliente', 'renta', 'segmento']

def getDataByMonth(dates, molist, clientfeat, prodlist):
    ids = dates.index[dates.fecha_dato.isin(molist)]
    print(ids[:10])
    print(molist)
    usecols = clientfeat + prodlist
    modata = pd.read_csv('C:/Users/SAMRUDDHI/data/train_ver2.csv', usecols=usecols,
                skiprows= range(1,ids[0]+1), nrows=len(ids), header=0)
    return modata
    
def getClientFeatures(mdata, prmdata):
    print('Employee index')
    #print(np.unique(mdata.ind_empleado, return_counts=True))
    mdata['employeeYN'] = mdata['ind_empleado']. \
                    map(lambda x: 0 if (x=='N' or x=='S') else 1)
    print(np.unique(mdata.employeeYN, return_counts=True))

    print('Seniority')
     # age type objectp
    if mdata.antiguedad.dtype != np.int64 and mdata.antiguedad.dtype != np.float64:
        mdata['antiguedad'] = mdata.antiguedad.str.strip()
        mdata['antiguedad'] = mdata.antiguedad.map(lambda x: None if x=='NA' 
                                                                else int(x))
    mdata.antiguedad[mdata.antiguedad<0] = mdata.antiguedad.max()
    mdata.antiguedad.fillna(mdata.antiguedad.median(), inplace=True)
    #print(mdata.antiguedad.describe())

    print('Age')
    if mdata.age.dtype != np.int64 and mdata.age.dtype != np.float64:
        mdata['age'] = mdata.age.str.strip()
        mdata['age'] = mdata.age.map(lambda x: None if x=='NA' else int(x))
    mdata.age.fillna(mdata.age.median(), inplace=True)
    #print(mdata.age.describe())
    
    print('Age categorical')
    mdata['agecateg'] = mdata.age.map(lambda x: '<18' if x <18 
                            else '18-25' if (x>=18 and x<25)
                            else '25-35' if (x>=25 and x<35)
                            else '35-45' if (x>=35 and x<45)
                            else '45-55' if (x>=45 and x<55)
                            else '55-65' if (x>=55 and x<65)
                            else '>65' if x>=65  else 'NA')
    print(np.unique(mdata.agecateg, return_counts=True))

    print('New customer index')
    print(np.unique(mdata.ind_nuevo, return_counts=True))
    
    print('Customer type')
    print(np.unique(mdata.indrel_1mes, return_counts=True))
    
    print('Customer relation type')
    #print(np.unique(mdata.tiprel_1mes, return_counts=True))
    mdata.tiprel_1mes.fillna('I', inplace=True)
    print(np.unique(mdata.tiprel_1mes, return_counts=True))

    print('Activity index')
    print(np.unique(mdata.ind_actividad_cliente, return_counts=True))
    
    print('Sex')
    #print(np.unique(mdata.sexo, return_counts=True))
    mdata['sexo'] = mdata['sexo'].map({'H':0, 'V':1, np.nan:1}).astype(int)
    print(np.unique(mdata.sexo, return_counts=True))

    print('Segmentation')
    #print(np.unique(mdata.segmento, return_counts=True))
    mdata.segmento.fillna('02 - PARTICULARES', inplace=True)
    #print(np.unique(mdata.segmento, return_counts=True))
    mdata.segmento = mdata.segmento.map(lambda x: x[:2])
    print(np.unique(mdata.segmento, return_counts=True))

    print('Deceased client')
    mdata.indfall.fillna('N', inplace=True)
    mdata['indfall'] = mdata['indfall'].map({'N':0, 'S':1}).astype(int)
    print(np.unique(mdata.indfall, return_counts=True)) 
    
    print('Province code')
    print(np.unique(mdata.cod_prov, return_counts=True))
    mdata.cod_prov.fillna(99, inplace=True) # Foreign clients

    print('Income') # 702435 non-null float64
    # Convert NA (string) to NaN
    mdata['renta'] = pd.to_numeric(mdata['renta'], errors='coerce')
    #print(mdata.renta.describe())
    #mdata['renta'].hist()
    #plt.hist(mdata.renta[mdata.renta<500000].dropna(), bins=20)
    #plt.show()
    print('Fill missing incomes with medians')
    for ac in mdata.agecateg.unique(): # agecateg
        for seg in mdata.segmento.unique(): # segment
            med = mdata[(mdata.agecateg==ac) & (mdata.segmento==seg)]['renta'] \
                                .dropna().median()
            mdata.loc[(mdata.renta.isnull()) & (mdata.agecateg==ac) & \
                        (mdata.segmento==seg), 'renta'] = med
            #print(ac, seg, med, mdata[(mdata.agecateg==ac) & \
            #                (mdata.segmento==seg)]['renta'].dropna().median())
        
    #plt.hist(mdata.renta[mdata.renta<500000].dropna(), bins=20)
    plt.show()
    
    Xclient = pd.concat([mdata[['ncodpers', 'employeeYN', 'sexo', 'age',
                                'antiguedad', 'indfall',
                                'ind_actividad_cliente', 'renta']], 
                        #pd.get_dummies(mdata['cod_prov']),
                        pd.get_dummies(mdata['tiprel_1mes'].apply(str)),
                        pd.get_dummies(mdata['segmento'].apply(str))],
                        axis=1)
    print(Xclient.columns)
    del mdata
    gc.collect()
    
    print('\nMerge with prev months prods...')
    
    X = pd.merge(Xclient, prmdata, how='left', on='ncodpers')
    print(X.shape)
    #print(X.info())
    print(X.head())
    #print(X.tail())
    # Fill products of new clients
    X.fillna(0, inplace=True)
    #print(X.info())
    
    return X
    
def getAddedProducts(mdata, prevmdata):

    intsec = np.intersect1d(mdata.ncodpers, prevmdata.ncodpers)
    print(intsec.size)
    print(np.unique(intsec).size)
    
    print('\nMerge...')
    mgd = pd.merge(mdata, prevmdata, how='left', on='ncodpers')
    print(mgd.shape)
    #print(mgd.info())
    print(mgd.head())
    #print(mgd.tail())
    mgd.fillna(0, inplace=True)
    #print(mgd.info())
      
    added = pd.DataFrame(mgd.ncodpers)
    #print(added.info())
    print(added.head())
    
    for i, pr in enumerate(targetprods):
        # Difference between this and previous month
        # 0: no change in product, 1: added product, -1: removed product
        #added[pr] = mgd.iloc[:, i+1] - mgd.iloc[:, i+25]
        added[pr] = mgd.loc[:, pr + '_x'] - mgd.loc[:, pr + '_y']
        display(mgd.loc[:, pr + '_x'])
        display(mgd.loc[:, pr + '_y'])
        
        # Consider only added products
        added.loc[added[pr] == -1, pr] = 0
    
    #print(added.info())
    print(added.head())
    print('Total added products')
    print(added.sum(axis=0))
    
    return added.drop(['ncodpers'], axis=1)
    
def getLaggedFeatures(mgd, molist):
    #print(mdata.shape)
    #mgd = mdata
    
    for mo in molist:
        print(mo)
        lagmodata = getDataByMonth(dates, [mo], ['ncodpers'], prods)
        print(lagmodata.shape)

        #intsec = np.intersect1d(mdata.ncodpers, lagmodata.ncodpers)
        #print(intsec.size)
        #print(np.unique(intsec).size)
        
        print('\nMerging lagged month (' +str(mo) + ')...')
        i = molist.index(mo)
        mgd = pd.merge(mgd, lagmodata, how='left', on='ncodpers', 
                                                    suffixes=[i, i+1])
        print(mgd.shape)
#         print(mgd.info())
        #print(mgd.head())
        #print(mgd.tail())
        mgd.fillna(0, inplace=True)
        print(mgd.info())
              
    print(mgd.info())
    print(mgd.head())
        
    return mgd

def apk(actual, predicted, k=7):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=7):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

#### Creating additional features and combining lagged month data into training data

In [3]:
print('\nLoading dates...')
dates = pd.read_csv('C:/Users/SAMRUDDHI/data/train_ver2.csv', usecols=['fecha_dato'], header=0)
print('Dates')
#print(dates.dtypes)
#print(dates.info())
print(dates.head())
#print(np.unique(dates, return_counts=True))
#print(dates.fecha_dato.unique())


Loading dates...
Dates
   fecha_dato
0  2015-01-28
1  2015-01-28
2  2015-01-28
3  2015-01-28
4  2015-01-28


In [4]:
dates.shape

(13647309, 1)

In [5]:
# get Jun 15 data as well as May 15 data

# Client features
thismonth = '2015-06-28' # '2016-05-28'
prevmonth = months[months.index(thismonth) - 1]
print('\nThis month: %s. Previous month: %s' % (thismonth, prevmonth))
 
mdata = getDataByMonth(dates, [thismonth], clientfeatures, prods)
prevmdata = getDataByMonth(dates, [prevmonth], ['ncodpers'], prods)
#mprod = mdata[prods]
display(mdata.head())
display(prevmdata.head())


This month: 2015-06-28. Previous month: 2015-05-28
Int64Index([3144384, 3144385, 3144386, 3144387, 3144388, 3144389, 3144390,
            3144391, 3144392, 3144393],
           dtype='int64')
['2015-06-28']


C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Int64Index([2512427, 2512428, 2512429, 2512430, 2512431, 2512432, 2512433,
            2512434, 2512435, 2512436],
           dtype='int64')
['2015-05-28']


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-06-28,16132,N,ES,V,48,1995-03-08,0.0,244,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-06-28,1063040,N,ES,V,25,2012-09-19,0.0,34,1.0,...,0,0,0,0,0,0,0,0.0,0.0,1
2,2015-06-28,1063041,N,ES,H,24,2012-09-19,0.0,34,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-06-28,1063042,N,ES,H,26,2012-09-19,0.0,34,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-06-28,1063043,N,ES,V,23,2012-09-19,0.0,34,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1061260,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,1061283,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,1061284,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,1061336,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,1061286,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
print('Get train data (this month client features + prev month prods')

#X = getClientFeatures(mdata[clientfeatures], prevmdata).drop(['ncodpers'], axis=1)
X = getClientFeatures(mdata[clientfeatures], prevmdata)

print('X shape {}'.format(X.shape))
print(X.head())

print('\nLagged features...')
lag1month = months[months.index(prevmonth) - 1]
lag2month = months[months.index(lag1month) - 1]
lag3month = months[months.index(lag2month) - 1]
lag4month = months[months.index(lag3month) - 1]
lag5month = months[months.index(lag4month) - 1]
lag6month = months[months.index(lag5month) - 1]
lag7month = months[months.index(lag6month) - 1]
# List of months used to get lagged product features
lagmonths = [lag1month, lag2month, lag3month, lag4month] #, lag5month]
                            #lag6month, lag7month]
print('Lagged months: ' + str(lagmonths))
X = getLaggedFeatures(X, lagmonths)
print('X shape {}'.format(X.shape))
print(X.head())

Get train data (this month client features + prev month prods
Employee index


<ipython-input-2-e0b23aca24cd>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata['employeeYN'] = mdata['ind_empleado']. \


(array([0, 1], dtype=int64), array([629768,   2342], dtype=int64))
Seniority


<ipython-input-2-e0b23aca24cd>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata['antiguedad'] = mdata.antiguedad.str.strip()
<ipython-input-2-e0b23aca24cd>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata['antiguedad'] = mdata.antiguedad.map(lambda x: None if x=='NA'
<ipython-input-2-e0b23aca24cd>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

Age


...

<ipython-input-2-e0b23aca24cd>:157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata['renta'] = pd.to_numeric(mdata['renta'], errors='coerce')


(array([ 1.,  2.,  3., ..., nan, nan, nan]), array([ 1723,  5312, 13916, ...,     1,     1,     1], dtype=int64))
Income
Fill missing incomes with medians


C:\Program Files\Anaconda\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Index(['ncodpers', 'employeeYN', 'sexo', 'age', 'antiguedad', 'indfall',
       'ind_actividad_cliente', 'renta', 'A', 'I', 'P', '01', '02', '03'],
      dtype='object')

Merge with prev months prods...
(632110, 38)
   ncodpers  employeeYN  sexo   age  antiguedad  indfall  \
0     16132           0     1  48.0       244.0        0   
1   1063040           0     1  25.0        34.0        0   
2   1063041           0     0  24.0        34.0        0   
3   1063042           0     0  26.0        34.0        0   
4   1063043           0     1  23.0        34.0        0   

   ind_actividad_cliente       renta  A  I  ...  ind_hip_fin_ult1  \
0                    0.0  160900.950  1  0  ...               0.0   
1                    1.0   74693.670  1  0  ...               0.0   
2                    1.0   35053.770  1  0  ...               0.0   
3                    0.0   72386.490  0  1  ...               0.0   
4                    1.0   87157.995  0  1  ...               0.0   

   ind_p

Int64Index([625457, 625458, 625459, 625460, 625461, 625462, 625463, 625464,
            625465, 625466],
           dtype='int64')
['2015-02-28']
(627394, 25)

Merging lagged month (2015-02-28)...
(632110, 110)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 632110 entries, 0 to 632109
Columns: 110 entries, ncodpers to ind_recibo_ult13
dtypes: float64(100), int32(2), int64(2), uint8(6)
memory usage: 505.2 MB
None
2015-01-28
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')
['2015-01-28']
(625457, 25)

Merging lagged month (2015-01-28)...
(632110, 134)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 632110 entries, 0 to 632109
Columns: 134 entries, ncodpers to ind_recibo_ult1
dtypes: float64(124), int32(2), int64(2), uint8(6)
memory usage: 620.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 632110 entries, 0 to 632109
Columns: 134 entries, ncodpers to ind_recibo_ult1
dtypes: float64(124), int32(2), int64(2), uint8(6)
memory usage: 620.9 MB
None
   ncodpers  emplo

#### Generate Output data as 'added' products between May-15 and June-15

In [7]:
# Remove codpers
X.drop(['ncodpers'], axis=1, inplace=True)

print('\nAdded products (targets)')
#print(mdata.ncodpers.describe())
#print(prevmdata.ncodpers.describe())

y = getAddedProducts(mdata[['ncodpers']+prods], prevmdata)
print('y shape {}'.format(y.shape))
print(y.values.sum()/y.size)
print(y[:5])

del mdata, prevmdata
gc.collect()


Added products (targets)
628603
628603

Merge...
(632110, 49)
   ncodpers  ind_ahor_fin_ult1_x  ind_aval_fin_ult1_x  ind_cco_fin_ult1_x  \
0     16132                    0                    0                   1   
1   1063040                    0                    0                   1   
2   1063041                    0                    0                   1   
3   1063042                    0                    0                   1   
4   1063043                    0                    0                   1   

   ind_cder_fin_ult1_x  ind_cno_fin_ult1_x  ind_ctju_fin_ult1_x  \
0                    0                   0                    0   
1                    0                   0                    0   
2                    0                   0                    0   
3                    0                   0                    0   
4                    0                   0                    0   

   ind_ctma_fin_ult1_x  ind_ctop_fin_ult1_x  ind_ctpp_fin_ult1_x  ...  

0         0
1         1
2         0
3         0
4         0
         ..
632105    0
632106    0
632107    0
632108    1
632109    0
Name: ind_recibo_ult1_x, Length: 632110, dtype: int64

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
632105    0.0
632106    0.0
632107    0.0
632108    1.0
632109    0.0
Name: ind_recibo_ult1_y, Length: 632110, dtype: float64

0         1
1         1
2         1
3         1
4         1
         ..
632105    1
632106    1
632107    0
632108    0
632109    1
Name: ind_cco_fin_ult1_x, Length: 632110, dtype: int64

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
632105    1.0
632106    1.0
632107    0.0
632108    0.0
632109    1.0
Name: ind_cco_fin_ult1_y, Length: 632110, dtype: float64

...

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
632105    0.0
632106    0.0
632107    0.0
632108    0.0
632109    0.0
Name: ind_ctpp_fin_ult1_y, Length: 632110, dtype: float64

0         0
1         0
2         0
3         0
4         0
         ..
632105    0
632106    0
632107    0
632108    0
632109    0
Name: ind_valo_fin_ult1_x, Length: 632110, dtype: int64

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
632105    0.0
632106    0.0
632107    0.0
632108    0.0
632109    0.0
Name: ind_valo_fin_ult1_y, Length: 632110, dtype: float64

   ncodpers  ind_recibo_ult1  ind_cco_fin_ult1  ind_nom_pens_ult1  \
0     16132              0.0               0.0                0.0   
1   1063040              0.0               0.0                0.0   
2   1063041              0.0               0.0                0.0   
3   1063042              0.0               0.0                0.0   
4   1063043              0.0               0.0                0.0   

   ind_nomina_ult1  ind_tjcr_fin_ult1  ind_ecue_fin_ult1  ind_cno_fin_ult1  \
0              0.0                0.0                0.0               0.0   
1              0.0                0.0                0.0               0.0   
2              0.0                0.0                0.0               0.0   
3              0.0                0.0                0.0               0.0   
4              0.0                0.0                0.0               0.0   

   ind_ctma_fin_ult1  ind_reca_fin_ult1  ind_ctop_fin_ult1  ind_ctpp_fin_ult1  \
0                0.0               

20

In [8]:
print('Training and validation sets')
# Create test set out of 20% samples
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.2,
                                                            random_state=0)                                                      
Xtrain = X
ytrain = y    
print(Xtrain.shape, ytrain.shape, Xval.shape, yval.shape)
del X, y
gc.collect()

print('Select only clients with added products')
addedprods= np.sum(ytrain, axis=1)
print(addedprods)
# Ensuring that records only with 'added' products are trained
Xtrain = Xtrain[addedprods!=0]
print(Xtrain)
ytrain = ytrain[addedprods!=0]
print(ytrain)
print(Xtrain.shape, ytrain.shape)

Training and validation sets
(632110, 133) (632110, 12) (126422, 133) (126422, 12)
Select only clients with added products
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
632105    0.0
632106    0.0
632107    0.0
632108    1.0
632109    0.0
Length: 632110, dtype: float64
        employeeYN  sexo   age  antiguedad  indfall  ind_actividad_cliente  \
142              0     1  28.0        34.0        0                    1.0   
161              0     1  37.0        34.0        0                    1.0   
170              0     0  40.0        34.0        0                    1.0   
178              0     1  46.0        34.0        0                    0.0   
188              0     0  41.0        34.0        0                    1.0   
...            ...   ...   ...         ...      ...                    ...   
632081           0     1  41.0       119.0        0                    1.0   
632082           0     0  38.0       119.0        0                 

In [21]:
# Collects all 1's(recommended prods), converts into prod labels and provides a list of recommended prods per record. 
targlist=[]
for row in yval.values:
    clientlist = []
    for i in range(yval.shape[1]):
        if row[i] == 1:
            clientlist.append(prodict[i])
    targlist.append(clientlist)

NameError: name 'yval' is not defined

#### Training Model and evaluating prediction score

In [10]:
######################## Train model #########################
print('\nTraining...')
clfdict = {}
probs = []
freq = ytrain.sum(axis=0)
for pr in targetprods:
    print(pr)
    clf = xgboost.XGBClassifier(max_depth=6, learning_rate = 0.05, 
                subsample = 0.9, colsample_bytree = 0.9, n_estimators=100,
                base_score = freq[pr]/Xtrain.shape[0], nthread=2)
    clfdict[pr] = clf
    clf.fit(Xtrain, ytrain.loc[:, pr])
    ypredv = clf.predict(Xval)
    probs.append(clf.predict_proba(Xval)[:, 1])
    
probs = np.array(probs).T
print(probs.shape) # m n

idsort7 = np.argsort(probs, axis=1)[:, :-8:-1] # ids of seven greatest probs
prlist = [[prodict[j] for j in irow] for irow in idsort7]

mapscore = mapk(targlist, prlist, 7)
print('MAP@7 score: %0.5f' % mapscore)


Training...
ind_recibo_ult1
ind_cco_fin_ult1
ind_nom_pens_ult1
ind_nomina_ult1
ind_tjcr_fin_ult1
ind_ecue_fin_ult1
ind_cno_fin_ult1
ind_ctma_fin_ult1
ind_reca_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_valo_fin_ult1
(126422, 12)
MAP@7 score: 0.05124


#### Transforming the test data and predicting on it

In [23]:
###################### Test predictions ###########################

print('\nTest predictions...')
testmonth = '2016-06-28'
prtmonth = '2016-05-28'
print('\nTest month: %s. Previous test month: %s' % (testmonth, prtmonth))

# del Xtrain, Xval, ytrain, yval
gc.collect()

tdata = pd.read_csv('C:/Users/SAMRUDDHI/data/test_ver2.csv', usecols=clientfeatures, header=0)
prtmdata = getDataByMonth(dates, [prtmonth], ['ncodpers'], prods)
#mprod = mdata[prods]
print(tdata.head())
print(prtmdata.head())

print('Get test data (test month client features + prev month prods')

Xtest = getClientFeatures(tdata[clientfeatures], prtmdata)

print('Lagged test months')
lag1month = months[months.index(prtmonth) - 1]
lag2month = months[months.index(lag1month) - 1]
lag3month = months[months.index(lag2month) - 1]
lag4month = months[months.index(lag3month) - 1]
lag5month = months[months.index(lag4month) - 1]
# List of months used to get lagged product features
lagtestmonths= [lag1month, lag2month, lag3month, lag4month] #, lag5month]
#lagtestmonths= ['2016-04-28', '2016-03-28']
print(lagtestmonths)
Xtest = getLaggedFeatures(Xtest, lagtestmonths)
         
tids = Xtest['ncodpers']

Xtest.drop(['ncodpers'], axis=1, inplace=True)

print(Xtest.shape)  # m n (929615 90)
print(Xtest.head())
del tdata
del prtmdata
gc.collect()

print('Prediction list...')
tclfdict = clfdict

testprobs = []
for pr in targetprods:
    print(pr)
    testprobs.append(tclfdict[pr].predict_proba(Xtest)[:, 1])

testprobs = np.array(testprobs).T
print(testprobs.shape)

print('Creating list of most probable products...')
idsort7 = np.argsort(testprobs, axis=1)[:, :-8:-1] # ids of seven greatest probs
predlist = [[prodict[j] for j in irow] for irow in idsort7]


Test predictions...

Test month: 2016-06-28. Previous test month: 2016-05-28


C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Int64Index([12715856, 12715857, 12715858, 12715859, 12715860, 12715861,
            12715862, 12715863, 12715864, 12715865],
           dtype='int64')
['2016-05-28']
   fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  fecha_alta  \
0  2016-06-28     15889            F              ES    V   56  1995-01-16   
1  2016-06-28   1170544            N              ES    H   36  2013-08-28   
2  2016-06-28   1170545            N              ES    V   22  2013-08-28   
3  2016-06-28   1170547            N              ES    H   22  2013-08-28   
4  2016-06-28   1170548            N              ES    H   22  2013-08-28   

   ind_nuevo  antiguedad  indrel  ... indext  conyuemp canal_entrada indfall  \
0          0         256       1  ...      N         N           KAT       N   
1          0          34       1  ...      N       NaN           KAT       N   
2          0          34       1  ...      N       NaN           KHE       N   
3          0          34       1  ...      N 

<ipython-input-20-e0b23aca24cd>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdata.antiguedad[mdata.antiguedad<0] = mdata.antiguedad.max()


(array(['18-25', '25-35', '35-45', '45-55', '55-65', '<18', '>65'],
      dtype=object), array([217249, 184802, 174297, 169177,  84556,   7286,  92248],
      dtype=int64))
New customer index
(array([0, 1], dtype=int64), array([903726,  25889], dtype=int64))
Customer type
(array([ 1.,  3., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), array([929565,     27,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1,      1,      1,      1,      1,      1,      1,      1,
            1], dtype=int64))
Customer relation type
(array(['A', 'I', 'P'], dtype=object), array([393622, 535966,     27], dtype=int64))
Activity index
(array([0, 1], dtype=int64), array([534276, 395339], dtype=int64))
Sex
(array([0, 1]), array([425095, 504520], dtype=int64))
Segmentation
(array(['01', '02', '03'], dtype=object), array([ 35961, 547626, 346028], dtype=i

(920904, 25)

Merging lagged month (2016-02-28)...
(929615, 110)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 929615 entries, 0 to 929614
Columns: 110 entries, ncodpers to ind_recibo_ult13
dtypes: float64(73), int32(2), int64(29), uint8(6)
memory usage: 742.9 MB
None
2016-01-28
Int64Index([9025333, 9025334, 9025335, 9025336, 9025337, 9025338, 9025339,
            9025340, 9025341, 9025342],
           dtype='int64')
['2016-01-28']
(916269, 25)

Merging lagged month (2016-01-28)...
(929615, 134)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 929615 entries, 0 to 929614
Columns: 134 entries, ncodpers to ind_recibo_ult1
dtypes: float64(97), int32(2), int64(29), uint8(6)
memory usage: 913.1 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 929615 entries, 0 to 929614
Columns: 134 entries, ncodpers to ind_recibo_ult1
dtypes: float64(97), int32(2), int64(29), uint8(6)
memory usage: 913.1 MB
None
   ncodpers  employeeYN  sexo  age  antiguedad  indfall  \
0     15889           

#### Submission

In [25]:
print('Submitting...')
subname = 'SantanderXGB10_lag4_12pr_nocv_md6.csv'
sub = tids.to_frame()
sub['added_products'] = np.array([' '.join(p) for p in predlist])
sub.to_csv(subname, index=False)

del sub
gc.collect()

Submitting...


140